In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### Request for Census Tract Level Population data from Census API

In [3]:
stateCode = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
            '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
             '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']

In [4]:
len(stateCode)

51

In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [6]:
def collectAll_county(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs5?get=NAME,'+ variables + '&for=county:*&in=state:*&key=250f58e35283d400e4185b197e3abe9021079afe'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

#### Total population: B01003_001E

In [7]:
year = '2014'
variables = 'B01003_001E'

In [8]:
pop_14 = collectAll_county(year,variables)
pop_14.head()

['NAME', 'B01003_001E', 'state', 'county']


,county_name,B01003_001E,state code,county code
0,"McPherson County, Nebraska",426,31,117
1,"Cheyenne County, Nebraska",10044,31,033
2,"Dawson County, Nebraska",24205,31,047
3,"Garfield County, Nebraska",1954,31,071
4,"Nance County, Nebraska",3667,31,125


In [9]:
pop_14.shape

(3220, 4)

#### 1.per capita income: B19301_001E

In [10]:
variables = 'B19301_001E'
df1 = collectAll_county(year,variables)

['NAME', 'B19301_001E', 'state', 'county']


In [11]:
df1.shape

(3220, 4)

In [12]:
df1 = df1.rename(columns = {'B19301_001E': 'per_capita_income'})
df1['fips'] = df1['state code'] + df1['county code']
df1.head()

,county_name,per_capita_income,state code,county code,fips
0,"McPherson County, Nebraska",25760,31,117,31117
1,"Cheyenne County, Nebraska",29728,31,033,31033
2,"Dawson County, Nebraska",21046,31,047,31047
3,"Garfield County, Nebraska",21939,31,071,31071
4,"Nance County, Nebraska",25499,31,125,31125


#### 2.Percent of Population Below Poverty Level: (C17002_002E+C17002_003E)/C17002_001E

In [13]:
variables = 'C17002_002E,C17002_003E,C17002_001E'
df2 = collectAll_county(year,variables)

['NAME', 'C17002_002E', 'C17002_003E', 'C17002_001E', 'state', 'county']


In [14]:
df2['C17002_002E'] = df2['C17002_002E'].astype('int')
df2['C17002_003E'] = df2['C17002_003E'].astype('int')
df2['C17002_001E'] = df2['C17002_001E'].astype('int')

In [15]:
df2['percent_below_poverty'] = ((df2['C17002_002E'] + df2['C17002_003E'])/df2['C17002_001E'])*100

In [16]:
df2['fips'] = df2['state code'] + df2['county code']
df2.head()

,county_name,C17002_002E,C17002_003E,C17002_001E,state code,county code,percent_below_poverty,fips
0,"McPherson County, Nebraska",16,33,426,31,117,11.502347,31117
1,"Cheyenne County, Nebraska",305,1035,9893,31,033,13.544931,31033
2,"Dawson County, Nebraska",1178,2126,23757,31,047,13.907480,31047
3,"Garfield County, Nebraska",54,175,1914,31,071,11.964472,31071
4,"Nance County, Nebraska",125,284,3576,31,125,11.437360,31125


#### 3.Percent of Population 25+ with Less than a 12th Grade Education: (B15002_003E,B15002_004E,B15002_005E,B15002_003E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E)/B15003_001E

In [17]:
def collectAll_county2(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs1?get=NAME,'+ variables + '&for=county:*&in=state:*'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

In [18]:
variables = 'B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E'
df3 = collectAll_county(year,variables)

['NAME', 'B15002_003E', 'B15002_004E', 'B15002_005E', 'B15002_006E', 'B15002_007E', 'B15002_008E', 'B15002_009E', 'B15002_010E', 'B15002_020E', 'B15002_021E', 'B15002_022E', 'B15002_023E', 'B15002_024E', 'B15002_025E', 'B15002_026E', 'B15002_027E', 'B15002_001E', 'state', 'county']


In [19]:
df3.shape

(3220, 20)

In [20]:
def changeInteger(value):
    if value != None:
        return int(value)
    else:
        return np.nan

In [21]:
df3['B15002_003E'] = df3['B15002_003E'].apply(changeInteger)
df3['B15002_004E'] = df3['B15002_004E'].apply(changeInteger)
df3['B15002_005E'] = df3['B15002_005E'].apply(changeInteger)
df3['B15002_006E'] = df3['B15002_006E'].apply(changeInteger)
df3['B15002_007E'] = df3['B15002_007E'].apply(changeInteger)
df3['B15002_008E'] = df3['B15002_008E'].apply(changeInteger)
df3['B15002_009E'] = df3['B15002_009E'].apply(changeInteger)
df3['B15002_010E'] = df3['B15002_010E'].apply(changeInteger)
df3['B15002_020E'] = df3['B15002_020E'].apply(changeInteger)
df3['B15002_021E'] = df3['B15002_021E'].apply(changeInteger)
df3['B15002_022E'] = df3['B15002_022E'].apply(changeInteger)
df3['B15002_023E'] = df3['B15002_023E'].apply(changeInteger)
df3['B15002_024E'] = df3['B15002_024E'].apply(changeInteger)
df3['B15002_025E'] = df3['B15002_025E'].apply(changeInteger)
df3['B15002_026E'] = df3['B15002_026E'].apply(changeInteger)
df3['B15002_027E'] = df3['B15002_027E'].apply(changeInteger)
df3['B15002_001E'] = df3['B15002_001E'].apply(changeInteger)

In [22]:
df3['percent_no_highschool_diploma'] = ((df3['B15002_003E']+df3['B15002_004E']+df3['B15002_005E']+df3['B15002_006E']+df3['B15002_007E']+df3['B15002_008E']+df3['B15002_009E']
                                       +df3['B15002_010E']+df3['B15002_020E']+df3['B15002_021E']+df3['B15002_022E']+df3['B15002_023E']+df3['B15002_024E']+df3['B15002_025E']+df3['B15002_003E']
                                       +df3['B15002_026E']+df3['B15002_027E'])/df3['B15002_001E'])*100

In [23]:
df3['fips'] = df3['state code'] + df3['county code']
df3.head()

,county_name,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,...,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E,state code,county code,percent_no_highschool_diploma,fips
0,"McPherson County, Nebraska",0,0,0,10,0,0,0,0,0,...,0,0,0,2,2,324,31,117,4.320988,31117
1,"Cheyenne County, Nebraska",3,0,3,48,18,76,13,59,18,...,9,31,24,22,23,6948,31,033,6.044905,31033
2,"Dawson County, Nebraska",224,232,493,223,134,196,191,222,152,...,195,234,294,190,107,15372,31,047,25.266719,31047
3,"Garfield County, Nebraska",3,0,0,9,3,18,3,4,0,...,19,0,14,2,1,1425,31,071,5.543860,31071
4,"Nance County, Nebraska",14,0,2,38,25,15,22,30,2,...,34,14,28,42,28,2586,31,125,12.258314,31125


#### 4.Percent of Population Living in Mobile Homes: (B25033_006E+B25033_007E+B25033_012E+B25033_013E)/B25033_001E

In [24]:
variables = 'B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E'
df4 = collectAll_county(year,variables)

['NAME', 'B25033_006E', 'B25033_007E', 'B25033_012E', 'B25033_013E', 'B25033_001E', 'state', 'county']


In [25]:
df4.shape

(3220, 8)

In [26]:
df4['B25033_006E'] = df4['B25033_006E'].apply(changeInteger)
df4['B25033_007E'] = df4['B25033_007E'].apply(changeInteger)
df4['B25033_012E'] = df4['B25033_012E'].apply(changeInteger)
df4['B25033_013E'] = df4['B25033_013E'].apply(changeInteger)
df4['B25033_001E'] = df4['B25033_001E'].apply(changeInteger)

In [27]:
df4['percent_mobile_homes'] = ((df4['B25033_006E'] + df4['B25033_007E'] + df4['B25033_012E'] + df4['B25033_013E'])/df4['B25033_001E'])*100

In [28]:
df4['fips'] = df4['state code'] + df4['county code']
df4.head()

,county_name,B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E,state code,county code,percent_mobile_homes,fips
0,"McPherson County, Nebraska",0,0,3,0,422,31,117,0.710900,31117
1,"Cheyenne County, Nebraska",216,0,119,0,9926,31,033,3.374975,31033
2,"Dawson County, Nebraska",1289,0,989,0,23857,31,047,9.548560,31047
3,"Garfield County, Nebraska",48,0,10,0,1921,31,071,3.019261,31071
4,"Nance County, Nebraska",43,0,56,0,3516,31,125,2.815700,31125


#### 5.Percent of Population with No Vehicle Available: (B25044_003E+B25044_010E)/B25044_001E

In [29]:
variables = 'B25044_003E,B25044_010E,B25044_001E'
df5 = collectAll_county(year,variables)

['NAME', 'B25044_003E', 'B25044_010E', 'B25044_001E', 'state', 'county']


In [30]:
df5.shape

(3220, 6)

In [31]:
df5['B25044_003E'] = df5['B25044_003E'].apply(changeInteger)
df5['B25044_010E'] = df5['B25044_010E'].apply(changeInteger)
df5['B25044_001E'] = df5['B25044_001E'].apply(changeInteger)

In [32]:
df5['percent_no_vehicle'] = ((df5['B25044_003E'] + df5['B25044_010E'])/df5['B25044_001E'])*100

In [33]:
df5['fips'] = df5['state code'] + df5['county code']
df5.head()

,county_name,B25044_003E,B25044_010E,B25044_001E,state code,county code,percent_no_vehicle,fips
0,"McPherson County, Nebraska",6,0,188,31,117,3.191489,31117
1,"Cheyenne County, Nebraska",29,174,4382,31,033,4.632588,31033
2,"Dawson County, Nebraska",80,255,8731,31,047,3.836903,31047
3,"Garfield County, Nebraska",10,16,870,31,071,2.988506,31071
4,"Nance County, Nebraska",25,50,1548,31,125,4.844961,31125


#### 6.Percent of Population Unemployed:sum(B21005_006E, B21005_007E, B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E)/B21005_001E

In [34]:
variables = 'B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E'
df6 = collectAll_county(year,variables)

['NAME', 'B21005_006E', 'B21005_007E', 'B21005_011E', 'B21005_012E', 'B21005_017E', 'B21005_022E', 'B21005_028E', 'B21005_029E', 'B21005_033E', 'B21005_034E', 'B21005_001E', 'state', 'county']


In [35]:
df6.shape

(3220, 14)

In [36]:
df6['B21005_006E'] = df6['B21005_006E'].apply(changeInteger)
df6['B21005_007E'] = df6['B21005_007E'].apply(changeInteger)
df6['B21005_011E'] = df6['B21005_011E'].apply(changeInteger)
df6['B21005_012E'] = df6['B21005_012E'].apply(changeInteger)
df6['B21005_017E'] = df6['B21005_017E'].apply(changeInteger)
df6['B21005_022E'] = df6['B21005_022E'].apply(changeInteger)
df6['B21005_028E'] = df6['B21005_028E'].apply(changeInteger)
df6['B21005_029E'] = df6['B21005_029E'].apply(changeInteger)
df6['B21005_033E'] = df6['B21005_033E'].apply(changeInteger)
df6['B21005_034E'] = df6['B21005_034E'].apply(changeInteger)
df6['B21005_001E'] = df6['B21005_001E'].apply(changeInteger)

In [37]:
df6['percent_unemployed_CDC'] = ((df6['B21005_006E']+df6['B21005_007E']+df6['B21005_011E']+df6['B21005_012E']+df6['B21005_017E']+df6['B21005_022E']+df6['B21005_028E']+df6['B21005_029E']+df6['B21005_033E']+df6['B21005_034E'])/df6['B21005_001E'])*100

In [38]:
df6['fips'] = df6['state code'] + df6['county code']
df6.head()

,county_name,B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E,state code,county code,percent_unemployed_CDC,fips
0,"McPherson County, Nebraska",0,0,0,13,0,1,0,0,2,6,247,31,117,8.906883,31117
1,"Cheyenne County, Nebraska",26,0,56,276,18,29,0,43,14,267,5999,31,033,12.152025,31033
2,"Dawson County, Nebraska",0,5,469,685,0,148,0,62,29,695,14006,31,047,14.943596,31047
3,"Garfield County, Nebraska",0,0,4,55,0,28,8,14,5,95,989,31,071,21.132457,31071
4,"Nance County, Nebraska",0,6,28,60,4,24,0,27,10,173,2105,31,125,15.771971,31125


#### 7.Percent of Population Living In Accommodations with Less Than 1 Room Per Person/Crowding: (B25014_005E+B25014_006E+B25014_007E+B25014_011E+B25014_012E+B25014_013E)/B25014_001E

In [39]:
variables = 'B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E'
df7 = collectAll_county(year,variables)

['NAME', 'B25014_005E', 'B25014_006E', 'B25014_007E', 'B25014_011E', 'B25014_012E', 'B25014_013E', 'B25014_001E', 'state', 'county']


In [40]:
df7.shape

(3220, 10)

In [41]:
df7['B25014_005E'] = df7['B25014_005E'].apply(changeInteger)
df7['B25014_006E'] = df7['B25014_006E'].apply(changeInteger)
df7['B25014_007E'] = df7['B25014_007E'].apply(changeInteger)
df7['B25014_011E'] = df7['B25014_011E'].apply(changeInteger)
df7['B25014_012E'] = df7['B25014_012E'].apply(changeInteger)
df7['B25014_013E'] = df7['B25014_013E'].apply(changeInteger)
df7['B25014_001E'] = df7['B25014_001E'].apply(changeInteger)

In [42]:
df7['percent_overcrowding'] = ((df7['B25014_005E']+df7['B25014_006E']+df7['B25014_007E']+df7['B25014_011E']+df7['B25014_012E']+df7['B25014_013E'])/df7['B25014_001E'])*100

In [43]:
df7['fips'] = df7['state code'] + df7['county code']
df7.head()

,county_name,B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E,state code,county code,percent_overcrowding,fips
0,"McPherson County, Nebraska",0,0,0,8,0,0,188,31,117,4.255319,31117
1,"Cheyenne County, Nebraska",37,0,0,8,6,0,4382,31,033,1.163852,31033
2,"Dawson County, Nebraska",203,29,2,187,4,0,8731,31,047,4.867713,31047
3,"Garfield County, Nebraska",7,0,0,0,0,0,870,31,071,0.804598,31071
4,"Nance County, Nebraska",0,0,0,9,5,0,1548,31,125,0.904393,31125


#### 8.Percent of Housing Units with 10+ Units in Structure: (B25024_007E+B25024_008E+B25024_009E)/B25024_001E

In [44]:
variables = 'B25024_007E,B25024_008E,B25024_009E,B25024_001E'
df8 = collectAll_county(year,variables)

['NAME', 'B25024_007E', 'B25024_008E', 'B25024_009E', 'B25024_001E', 'state', 'county']


In [45]:
df8.shape

(3220, 7)

In [46]:
df8['B25024_007E'] = df8['B25024_007E'].apply(changeInteger)
df8['B25024_008E'] = df8['B25024_008E'].apply(changeInteger)
df8['B25024_009E'] = df8['B25024_009E'].apply(changeInteger)
df8['B25024_001E'] = df8['B25024_001E'].apply(changeInteger)

In [47]:
df8['percent_multi_unit_housing'] = ((df8['B25024_007E']+df8['B25024_008E']+df8['B25024_009E'])/df8['B25024_001E'])*100

In [48]:
df8['fips'] = df8['state code'] + df8['county code']
df8.head()

,county_name,B25024_007E,B25024_008E,B25024_009E,B25024_001E,state code,county code,percent_multi_unit_housing,fips
0,"McPherson County, Nebraska",0,0,0,244,31,117,0.000000,31117
1,"Cheyenne County, Nebraska",48,108,50,4897,31,033,4.206657,31033
2,"Dawson County, Nebraska",163,235,161,10135,31,047,5.515540,31047
3,"Garfield County, Nebraska",34,0,0,1171,31,071,2.903501,31071
4,"Nance County, Nebraska",10,23,0,1820,31,125,1.813187,31125


#### 9.Percent Of Population 65+: (B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_049E)/B01001_001E

In [49]:
variables = 'B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E'
df9 = collectAll_county(year,variables)

['NAME', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E', 'B01001_001E', 'state', 'county']


In [50]:
df9.shape

(3220, 16)

In [51]:
df9['B01001_020E'] = df9['B01001_020E'].apply(changeInteger)
df9['B01001_021E'] = df9['B01001_021E'].apply(changeInteger)
df9['B01001_022E'] = df9['B01001_022E'].apply(changeInteger)
df9['B01001_023E'] = df9['B01001_023E'].apply(changeInteger)
df9['B01001_024E'] = df9['B01001_024E'].apply(changeInteger)
df9['B01001_025E'] = df9['B01001_025E'].apply(changeInteger)
df9['B01001_044E'] = df9['B01001_044E'].apply(changeInteger)
df9['B01001_045E'] = df9['B01001_045E'].apply(changeInteger)
df9['B01001_046E'] = df9['B01001_046E'].apply(changeInteger)
df9['B01001_047E'] = df9['B01001_047E'].apply(changeInteger)
df9['B01001_048E'] = df9['B01001_048E'].apply(changeInteger)
df9['B01001_049E'] = df9['B01001_049E'].apply(changeInteger)
df9['B01001_001E'] = df9['B01001_001E'].apply(changeInteger)

In [52]:
df9['percent_age_65_and_older'] = ((df9['B01001_020E']+df9['B01001_021E']+df9['B01001_022E']+df9['B01001_023E']+df9['B01001_024E']+df9['B01001_025E']+df9['B01001_044E']
                                  +df9['B01001_045E']+df9['B01001_046E']+df9['B01001_047E']+df9['B01001_048E']+df9['B01001_049E'])/df9['B01001_001E'])*100

In [53]:
df9['fips'] = df9['state code'] + df9['county code']
df9.head()

,county_name,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E,state code,county code,percent_age_65_and_older,fips
0,"McPherson County, Nebraska",8,2,5,6,10,12,10,2,2,14,6,12,426,31,117,20.892019,31117
1,"Cheyenne County, Nebraska",81,146,127,161,96,75,108,109,171,266,144,133,10044,31,033,16.099164,31033
2,"Dawson County, Nebraska",151,286,360,287,181,198,149,313,442,459,297,261,24205,31,047,13.980583,31047
3,"Garfield County, Nebraska",17,28,81,66,36,15,33,51,52,60,49,35,1954,31,071,26.765609,31071
4,"Nance County, Nebraska",45,24,79,64,50,40,34,70,62,89,71,89,3667,31,125,19.552768,31125


#### 10.Percent Of Population Who Do Not Speak English: B99163_005E/B99163_001E

In [54]:
variables = 'B99163_005E,B99163_001E'
df10 = collectAll_county(year,variables)

['NAME', 'B99163_005E', 'B99163_001E', 'state', 'county']


In [55]:
df10.shape

(3220, 5)

In [56]:
df10['B99163_005E'] = df10['B99163_005E'].apply(changeInteger)
df10['B99163_001E'] = df10['B99163_001E'].apply(changeInteger)

In [57]:
df10['percent_limited_english_abilities'] = (df10['B99163_005E']/df10['B99163_001E'])*100

In [58]:
df10['fips'] = df10['state code'] + df10['county code']
df10.head()

,county_name,B99163_005E,B99163_001E,state code,county code,percent_limited_english_abilities,fips
0,"McPherson County, Nebraska",0,392,31,117,0.000000,31117
1,"Cheyenne County, Nebraska",497,9418,31,033,5.277129,31033
2,"Dawson County, Nebraska",7125,22355,31,047,31.872064,31047
3,"Garfield County, Nebraska",25,1870,31,071,1.336898,31071
4,"Nance County, Nebraska",66,3451,31,125,1.912489,31125


#### 11.Percent of Population 17 Years of Age and Under: (B01001_003E+B01001_004E+B01001_005E+B01001_006E+B01001_027E+B01001_028E+B01001_029E+B01001_030E)/B01003_001E

In [59]:
variables = 'B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E'
df11 = collectAll_county(year,variables)

['NAME', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01003_001E', 'state', 'county']


In [60]:
df11.shape

(3220, 12)

In [61]:
df11['B01001_003E'] = df11['B01001_003E'].apply(changeInteger)
df11['B01001_004E'] = df11['B01001_004E'].apply(changeInteger)
df11['B01001_005E'] = df11['B01001_005E'].apply(changeInteger)
df11['B01001_006E'] = df11['B01001_006E'].apply(changeInteger)
df11['B01001_027E'] = df11['B01001_027E'].apply(changeInteger)
df11['B01001_028E'] = df11['B01001_028E'].apply(changeInteger)
df11['B01001_029E'] = df11['B01001_029E'].apply(changeInteger)
df11['B01001_030E'] = df11['B01001_030E'].apply(changeInteger)
df11['B01003_001E'] = df11['B01003_001E'].apply(changeInteger)

In [62]:
df11['percent_age_17_and_younger'] = ((df11['B01001_003E']+df11['B01001_004E']+df11['B01001_005E']+df11['B01001_006E']+df11['B01001_027E']+df11['B01001_028E']+df11['B01001_029E']+df11['B01001_030E'])/df11['B01003_001E'])*100

In [63]:
df11['fips'] = df11['state code'] + df11['county code']
df11.head()

,county_name,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E,state code,county code,percent_age_17_and_younger,fips
0,"McPherson County, Nebraska",21,11,6,12,13,11,12,4,426,31,117,21.126761,31117
1,"Cheyenne County, Nebraska",310,466,294,196,316,253,400,190,10044,31,033,24.143767,31033
2,"Dawson County, Nebraska",989,1069,820,574,861,1058,874,570,24205,31,047,28.155340,31047
3,"Garfield County, Nebraska",42,59,74,44,42,101,43,37,1954,31,071,22.620266,31071
4,"Nance County, Nebraska",115,118,138,77,101,96,125,75,3667,31,125,23.043360,31125


#### 12.Percent Minority: 1-(B03002_003E/B03002_001E)

In [64]:
variables = 'B03002_003E,B03002_001E'
df12 = collectAll_county(year,variables)

['NAME', 'B03002_003E', 'B03002_001E', 'state', 'county']


In [65]:
df12.shape

(3220, 5)

In [66]:
df12['B03002_003E'] = df12['B03002_003E'].apply(changeInteger)
df12['B03002_001E'] = df12['B03002_001E'].apply(changeInteger)

In [67]:
df12['percent_minorities'] = (1 - (df12['B03002_003E']/df12['B03002_001E']))*100

In [68]:
df12['fips'] = df12['state code'] + df12['county code']
df12.head()

,county_name,B03002_003E,B03002_001E,state code,county code,percent_minorities,fips
0,"McPherson County, Nebraska",418,426,31,117,1.877934,31117
1,"Cheyenne County, Nebraska",9060,10044,31,033,9.796894,31033
2,"Dawson County, Nebraska",15038,24205,31,047,37.872340,31047
3,"Garfield County, Nebraska",1917,1954,31,071,1.893552,31071
4,"Nance County, Nebraska",3569,3667,31,125,2.672484,31125


#### 13.Percent of Population with a Disability: (B18101_004E+B18101_007E+B18101_010E+B18101_013E+B18101_016E+B18101_019E)/B18101_002E + (B18101_023E+B18101_026E+B18101_029E+B18101_032E+B18101_035E+B18101_038E)/B18101_021E

In [69]:
variables = 'B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E'
df13 = collectAll_county(year,variables)

['NAME', 'B18101_004E', 'B18101_007E', 'B18101_010E', 'B18101_013E', 'B18101_016E', 'B18101_019E', 'B18101_002E', 'B18101_023E', 'B18101_026E', 'B18101_029E', 'B18101_032E', 'B18101_035E', 'B18101_038E', 'B18101_021E', 'state', 'county']


In [70]:
df13.shape

(3220, 17)

In [71]:
df13['B18101_004E'] = df13['B18101_004E'].apply(changeInteger)
df13['B18101_007E'] = df13['B18101_007E'].apply(changeInteger)
df13['B18101_010E'] = df13['B18101_010E'].apply(changeInteger)
df13['B18101_013E'] = df13['B18101_013E'].apply(changeInteger)
df13['B18101_016E'] = df13['B18101_016E'].apply(changeInteger)
df13['B18101_002E'] = df13['B18101_002E'].apply(changeInteger)
df13['B18101_023E'] = df13['B18101_023E'].apply(changeInteger)
df13['B18101_026E'] = df13['B18101_026E'].apply(changeInteger)
df13['B18101_029E'] = df13['B18101_029E'].apply(changeInteger)
df13['B18101_032E'] = df13['B18101_032E'].apply(changeInteger)
df13['B18101_035E'] = df13['B18101_035E'].apply(changeInteger)
df13['B18101_038E'] = df13['B18101_038E'].apply(changeInteger)
df13['B18101_021E'] = df13['B18101_021E'].apply(changeInteger)

In [72]:
df13['percent_disabled'] = ((df13['B18101_004E']+df13['B18101_007E']+df13['B18101_010E']+df13['B18101_013E']+df13['B18101_016E'])/df13['B18101_002E'] + (df13['B18101_023E']+df13['B18101_026E']+df13['B18101_029E']+df13['B18101_032E']+df13['B18101_035E']+df13['B18101_038E'])/df13['B18101_021E'])*100

In [73]:
df13['fips'] = df13['state code'] + df13['county code']
df13.head()

,county_name,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E,state code,county code,percent_disabled,fips
0,"McPherson County, Nebraska",0,0,5,6,4,23,214,0,5,8,12,2,8,212,31,117,23.518780,31117
1,"Cheyenne County, Nebraska",11,31,120,243,127,135,4978,0,10,7,168,100,183,4955,31,033,20.132028,31033
2,"Dawson County, Nebraska",0,139,151,527,230,319,12034,6,106,161,500,175,420,11902,31,047,20.194216,31047
3,"Garfield County, Nebraska",0,6,26,54,30,45,952,0,10,7,59,13,36,969,31,071,25.084771,31071
4,"Nance County, Nebraska",14,17,12,107,38,93,1802,0,0,6,96,37,115,1774,31,125,24.750778,31125


#### 14.Percent of Population Living in Group Quarters: B26001_001E/B01003_001E

In [74]:
variables = 'B26001_001E,B01003_001E'
df14 = collectAll_county(year,variables)

['NAME', 'B26001_001E', 'B01003_001E', 'state', 'county']


In [75]:
df14.shape

(3220, 5)

In [76]:
df14['B26001_001E'] = df14['B26001_001E'].apply(changeInteger)
df14['B01003_001E'] = df14['B01003_001E'].apply(changeInteger)

In [77]:
df14['percent_institutionalized_in_group_quarters'] = (df14['B26001_001E']/df14['B01003_001E'])*100

In [78]:
df14['fips'] = df14['state code'] + df14['county code']
df14.head()

,county_name,B26001_001E,B01003_001E,state code,county code,percent_institutionalized_in_group_quarters,fips
0,"McPherson County, Nebraska",4,426,31,117,0.938967,31117
1,"Cheyenne County, Nebraska",118,10044,31,033,1.174831,31033
2,"Dawson County, Nebraska",348,24205,31,047,1.437719,31047
3,"Garfield County, Nebraska",33,1954,31,071,1.688843,31071
4,"Nance County, Nebraska",151,3667,31,125,4.117807,31125


#### 15.Percent of Children Living in Single Parent Households: (B09008_010E+B09008_011E+B09008_012E)/B09008_001E

In [79]:
variables = 'B09008_010E,B09008_011E,B09008_012E,B09008_001E'
df15 = collectAll_county(year,variables)

['NAME', 'B09008_010E', 'B09008_011E', 'B09008_012E', 'B09008_001E', 'state', 'county']


In [80]:
df15['B09008_010E'] = df15['B09008_010E'].apply(changeInteger)
df15['B09008_011E'] = df15['B09008_011E'].apply(changeInteger)
df15['B09008_012E'] = df15['B09008_012E'].apply(changeInteger)
df15['B09008_001E'] = df15['B09008_001E'].apply(changeInteger)

In [81]:
df15['percent_single_parent_households_CDC'] = ((df15['B09008_010E'] + df15['B09008_011E']+df15['B09008_012E'])/df15['B09008_001E'])*100

In [82]:
df15['fips'] = df15['state code'] + df15['county code']
df15.head()

,county_name,B09008_010E,B09008_011E,B09008_012E,B09008_001E,state code,county code,percent_single_parent_households_CDC,fips
0,"McPherson County, Nebraska",2,4,0,90,31,117,6.666667,31117
1,"Cheyenne County, Nebraska",162,403,1,2424,31,033,23.349835,31033
2,"Dawson County, Nebraska",340,1047,0,6791,31,047,20.424091,31047
3,"Garfield County, Nebraska",14,56,0,442,31,071,15.837104,31071
4,"Nance County, Nebraska",19,89,0,845,31,125,12.781065,31125


### age 18 to 34
#### (B01001_007E+B01001_008E+B01001_009E+B01001_010E+B01001_011E+B01001_012E+B01001_031E+B01001_032E+B01001_033E+B01001_034E+B01001_035E+B01001_036E)/B01001_001E

In [83]:
variables = 'B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E'
df16 = collectAll_county(year,variables)

['NAME', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_001E', 'state', 'county']


In [84]:
df16['B01001_007E'] = df16['B01001_007E'].apply(changeInteger)
df16['B01001_008E'] = df16['B01001_008E'].apply(changeInteger)
df16['B01001_009E'] = df16['B01001_009E'].apply(changeInteger)
df16['B01001_010E'] = df16['B01001_010E'].apply(changeInteger)
df16['B01001_011E'] = df16['B01001_011E'].apply(changeInteger)
df16['B01001_012E'] = df16['B01001_012E'].apply(changeInteger)
df16['B01001_031E'] = df16['B01001_031E'].apply(changeInteger)
df16['B01001_032E'] = df16['B01001_032E'].apply(changeInteger)
df16['B01001_033E'] = df16['B01001_033E'].apply(changeInteger)
df16['B01001_034E'] = df16['B01001_034E'].apply(changeInteger)
df16['B01001_035E'] = df16['B01001_035E'].apply(changeInteger)
df16['B01001_036E'] = df16['B01001_036E'].apply(changeInteger)
df16['B01001_001E'] = df16['B01001_001E'].apply(changeInteger)

In [85]:
df16['percent_age_18_to_34'] = ((df16['B01001_007E'] + df16['B01001_008E'] +df16['B01001_009E'] +df16['B01001_010E'] +df16['B01001_011E']+df16['B01001_012E']+df16['B01001_031E']+df16['B01001_032E']+df16['B01001_033E']+df16['B01001_034E']+df16['B01001_035E']+df16['B01001_036E'])/df16['B01001_001E'])*100

In [86]:
df16['fips'] = df16['state code'] + df16['county code']
df16.head()

,county_name,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E,state code,county code,percent_age_18_to_34,fips
0,"McPherson County, Nebraska",3,0,0,1,17,7,3,0,0,5,15,5,426,31,117,13.145540,31117
1,"Cheyenne County, Nebraska",106,43,61,168,352,304,104,16,28,145,335,351,10044,31,033,20.041816,31033
2,"Dawson County, Nebraska",318,172,103,428,819,755,299,115,103,480,705,726,24205,31,047,20.751911,31047
3,"Garfield County, Nebraska",0,9,0,29,51,57,13,0,0,36,36,42,1954,31,071,13.971341,31071
4,"Nance County, Nebraska",17,21,29,53,96,100,48,5,17,46,80,89,3667,31,125,16.389419,31125


### Age 35 to 44
#### (B01001_013E+B01001_014E+B01001_037E+B01001_038E)/B01001_001E

In [87]:
variables = 'B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E'
df17 = collectAll_county(year,variables)

['NAME', 'B01001_013E', 'B01001_014E', 'B01001_037E', 'B01001_038E', 'B01001_001E', 'state', 'county']


In [88]:
df17['B01001_013E'] = df17['B01001_013E'].apply(changeInteger)
df17['B01001_014E'] = df17['B01001_014E'].apply(changeInteger)
df17['B01001_037E'] = df17['B01001_037E'].apply(changeInteger)
df17['B01001_038E'] = df17['B01001_038E'].apply(changeInteger)
df17['B01001_001E'] = df17['B01001_001E'].apply(changeInteger)

In [89]:
df17['percent_age_35_to_44'] = ((df17['B01001_013E']+df17['B01001_014E'] +df17['B01001_037E']+df17['B01001_038E'])/df17['B01001_001E'])*100

In [90]:
df17['fips'] = df17['state code'] + df17['county code']
df17.head()

,county_name,B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E,state code,county code,percent_age_35_to_44,fips
0,"McPherson County, Nebraska",8,18,16,4,426,31,117,10.798122,31117
1,"Cheyenne County, Nebraska",383,251,235,303,10044,31,033,11.668658,31033
2,"Dawson County, Nebraska",647,817,613,809,24205,31,047,11.923156,31047
3,"Garfield County, Nebraska",44,20,42,33,1954,31,071,7.113613,31071
4,"Nance County, Nebraska",48,121,65,129,3667,31,125,9.899100,31125


### age 45 to 54
#### (B01001_015E+B01001_016E+B01001_039E+B01001_040E)/B01001_001E

In [91]:
variables = 'B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E'
df18 = collectAll_county(year,variables)

['NAME', 'B01001_015E', 'B01001_016E', 'B01001_039E', 'B01001_040E', 'B01001_001E', 'state', 'county']


In [92]:
df18['B01001_015E'] = df18['B01001_015E'].apply(changeInteger)
df18['B01001_016E'] = df18['B01001_016E'].apply(changeInteger)
df18['B01001_039E'] = df18['B01001_039E'].apply(changeInteger)
df18['B01001_040E'] = df18['B01001_040E'].apply(changeInteger)
df18['B01001_001E'] = df18['B01001_001E'].apply(changeInteger)

In [93]:
df18['percent_age_45_to_54'] = ((df18['B01001_015E']+df18['B01001_016E'] +df18['B01001_039E']+df18['B01001_040E'])/df18['B01001_001E'])*100

In [94]:
df18['fips'] = df18['state code'] + df18['county code']
df18.head()

,county_name,B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E,state code,county code,percent_age_45_to_54,fips
0,"McPherson County, Nebraska",21,7,30,28,426,31,117,20.187793,31117
1,"Cheyenne County, Nebraska",325,384,363,417,10044,31,033,14.824771,31033
2,"Dawson County, Nebraska",803,851,682,762,24205,31,047,12.799008,31047
3,"Garfield County, Nebraska",50,80,58,84,1954,31,071,13.920164,31071
4,"Nance County, Nebraska",128,166,114,142,3667,31,125,14.998636,31125


### age 55 to 64
#### (B01001_017E+B01001_018E+B01001_019E+B01001_041E+B01001_042E+B01001_043E)/B01001_001E

In [95]:
variables = 'B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E'
df19 = collectAll_county(year,variables)

['NAME', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 'B01001_001E', 'state', 'county']


In [96]:
df19['B01001_017E'] = df19['B01001_017E'].apply(changeInteger)
df19['B01001_018E'] = df19['B01001_018E'].apply(changeInteger)
df19['B01001_019E'] = df19['B01001_019E'].apply(changeInteger)
df19['B01001_041E'] = df19['B01001_041E'].apply(changeInteger)
df19['B01001_042E'] = df19['B01001_042E'].apply(changeInteger)
df19['B01001_043E'] = df19['B01001_043E'].apply(changeInteger)
df19['B01001_001E'] = df19['B01001_001E'].apply(changeInteger)

In [97]:
df19['percent_age_55_to_64'] = ((df19['B01001_017E']+df19['B01001_018E'] +df19['B01001_019E']+df19['B01001_041E']+df19['B01001_042E']+df19['B01001_043E'])/df19['B01001_001E'])*100

In [98]:
df19['fips'] = df19['state code'] + df19['county code']
df19.head()

,county_name,B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E,state code,county code,percent_age_55_to_64,fips
0,"McPherson County, Nebraska",30,4,5,10,7,3,426,31,117,13.849765,31117
1,"Cheyenne County, Nebraska",399,142,151,362,129,145,10044,31,033,13.221824,31033
2,"Dawson County, Nebraska",981,297,295,743,270,413,24205,31,047,12.390002,31047
3,"Garfield County, Nebraska",68,57,42,85,29,24,1954,31,071,15.609007,31071
4,"Nance County, Nebraska",150,53,108,142,70,68,3667,31,125,16.116717,31125


### Combine 19 dataframes together

In [99]:
df = df1[['fips','per_capita_income']].merge(df2[['fips', 'percent_below_poverty']], on = 'fips', how = 'left')

In [100]:
df = df.merge(df3[['fips', 'percent_no_highschool_diploma']], on = 'fips', how = 'left')
df = df.merge(df4[['fips', 'percent_mobile_homes']], on = 'fips', how = 'left')
df = df.merge(df5[['fips', 'percent_no_vehicle']], on = 'fips', how = 'left')
df = df.merge(df6[['fips', 'percent_unemployed_CDC']], on = 'fips', how = 'left')
df = df.merge(df7[['fips', 'percent_overcrowding']], on = 'fips', how = 'left')
df = df.merge(df8[['fips', 'percent_multi_unit_housing']], on = 'fips', how = 'left')
df = df.merge(df9[['fips', 'percent_age_65_and_older']], on = 'fips', how = 'left')
df = df.merge(df10[['fips', 'percent_limited_english_abilities']], on = 'fips', how = 'left')
df = df.merge(df11[['fips', 'percent_age_17_and_younger']], on = 'fips', how = 'left')
df = df.merge(df12[['fips', 'percent_minorities']], on = 'fips', how = 'left')
df = df.merge(df13[['fips', 'percent_disabled']], on = 'fips', how = 'left')
df = df.merge(df14[['fips', 'percent_institutionalized_in_group_quarters']], on = 'fips', how = 'left')
df = df.merge(df15[['fips', 'percent_single_parent_households_CDC']], on = 'fips', how = 'left')
df = df.merge(df16[['fips', 'percent_age_18_to_34']], on = 'fips', how = 'left')
df = df.merge(df17[['fips', 'percent_age_35_to_44']], on = 'fips', how = 'left')
df = df.merge(df18[['fips', 'percent_age_45_to_54']], on = 'fips', how = 'left')
df = df.merge(df19[['fips', 'percent_age_55_to_64']], on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,31117,25760,11.502347,4.320988,0.710900,3.191489,8.906883,4.255319,0.000000,20.892019,0.000000,21.126761,1.877934,23.518780,0.938967,6.666667,13.145540,10.798122,20.187793,13.849765
1,31033,29728,13.544931,6.044905,3.374975,4.632588,12.152025,1.163852,4.206657,16.099164,5.277129,24.143767,9.796894,20.132028,1.174831,23.349835,20.041816,11.668658,14.824771,13.221824
2,31047,21046,13.907480,25.266719,9.548560,3.836903,14.943596,4.867713,5.515540,13.980583,31.872064,28.155340,37.872340,20.194216,1.437719,20.424091,20.751911,11.923156,12.799008,12.390002
3,31071,21939,11.964472,5.543860,3.019261,2.988506,21.132457,0.804598,2.903501,26.765609,1.336898,22.620266,1.893552,25.084771,1.688843,15.837104,13.971341,7.113613,13.920164,15.609007
4,31125,25499,11.437360,12.258314,2.815700,4.844961,15.771971,0.904393,1.813187,19.552768,1.912489,23.043360,2.672484,24.750778,4.117807,12.781065,16.389419,9.899100,14.998636,16.116717


In [101]:
df['percent_age_18_to_64'] = 100 - df['percent_age_17_and_younger'] - df['percent_age_65_and_older']

In [102]:
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,...,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64,percent_age_18_to_64
0,31117,25760,11.502347,4.320988,0.710900,3.191489,8.906883,4.255319,0.000000,20.892019,...,21.126761,1.877934,23.518780,0.938967,6.666667,13.145540,10.798122,20.187793,13.849765,57.981221
1,31033,29728,13.544931,6.044905,3.374975,4.632588,12.152025,1.163852,4.206657,16.099164,...,24.143767,9.796894,20.132028,1.174831,23.349835,20.041816,11.668658,14.824771,13.221824,59.757069
2,31047,21046,13.907480,25.266719,9.548560,3.836903,14.943596,4.867713,5.515540,13.980583,...,28.155340,37.872340,20.194216,1.437719,20.424091,20.751911,11.923156,12.799008,12.390002,57.864078
3,31071,21939,11.964472,5.543860,3.019261,2.988506,21.132457,0.804598,2.903501,26.765609,...,22.620266,1.893552,25.084771,1.688843,15.837104,13.971341,7.113613,13.920164,15.609007,50.614125
4,31125,25499,11.437360,12.258314,2.815700,4.844961,15.771971,0.904393,1.813187,19.552768,...,23.043360,2.672484,24.750778,4.117807,12.781065,16.389419,9.899100,14.998636,16.116717,57.403872


In [103]:
df = df.sort_values('fips')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,...,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64,percent_age_18_to_64
2072,01001,24644,12.805235,14.776585,18.839361,5.324074,19.564231,2.610323,3.753734,13.278076,...,25.779890,23.764872,29.967308,0.801654,25.176007,20.648941,14.301001,14.999275,10.992818,60.942034
809,01003,26851,13.776798,11.269857,13.924091,3.068795,23.230896,1.366038,17.987363,17.667948,...,22.586229,16.818598,24.800071,1.365550,22.370560,19.142805,12.717241,14.140844,13.744933,59.745823
2063,01005,17350,24.084245,26.598004,25.439175,8.769820,33.635885,1.946419,1.090172,15.413548,...,21.615841,53.888418,36.558944,10.579299,47.117025,23.124009,12.592647,14.333124,12.920830,62.970611
2064,01007,18110,17.042654,23.018555,30.668501,4.224357,33.282884,0.240181,1.814135,14.165894,...,21.643932,25.237275,29.863265,6.957136,26.557040,22.835827,13.464000,16.019953,11.870392,64.190173
2070,01009,20501,17.283605,22.769640,23.794088,3.931403,26.092869,1.643260,0.900788,15.911181,...,24.073207,11.839709,30.964474,0.992280,22.392151,19.866424,13.109550,14.249284,12.790355,60.015613


In [104]:
df = df.drop(['percent_age_17_and_younger'], axis = 1)

In [105]:
df.to_csv('Social_Economic_2014.csv')

In [87]:
suicide_2017.head()

,county,fips,SuicideDeathRate
0,"Abbeville County, SC",45001,17.443705
1,"Acadia Parish, LA",22001,15.976993
2,"Accomack County, VA",51001,13.618293
3,"Ada County, ID",16001,18.824601
4,"Adair County, IA",19001,11.463289


In [88]:
df = df.merge(suicide_2017, on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,county,SuicideDeathRate
0,40045,27272,0.171555,0.124121,0.141763,0.028923,0.225352,0.010462,0.005712,0.209410,0.045809,0.236714,0.130912,0.481632,0.012921,0.285273,"Ellis County, OK",22.410690
1,40051,27334,0.124159,0.119022,0.145647,0.033480,0.208998,0.021679,0.023391,0.151570,0.037375,0.244448,0.180015,0.617015,0.012640,0.219735,"Grady County, OK",22.775042
2,40119,22785,0.263145,0.089728,0.100425,0.056900,0.321887,0.033154,0.111002,0.116006,0.062704,0.193678,0.226592,0.529113,0.099003,0.218812,"Payne County, OK",14.710389
3,40125,22284,0.169700,0.135085,0.122731,0.052806,0.243826,0.024648,0.032731,0.155835,0.043118,0.243011,0.270296,0.647418,0.045424,0.256268,"Pottawatomie County, OK",16.614516
4,40037,24056,0.151820,0.137377,0.208241,0.036480,0.227509,0.035571,0.024049,0.167859,0.026556,0.242246,0.232415,0.494830,0.011721,0.262649,"Creek County, OK",13.946223


In [89]:
df.shape

(3220, 18)

In [90]:
df.to_csv('Social_Economic_Suicide_2017.csv')